In [2]:
import pandas as pd

df = pd.read_csv('data/hcvdat0.csv')
print("\nFirst 5 rows of the dataframe: ")
df.head()


First 5 rows of the dataframe: 


,Unnamed: 0,Category,Age,Sex,ALB,ALP,ALT,AST,BIL,CHE,CHOL,CREA,GGT,PROT
0,1,0=Blood Donor,32,m,38.5,52.5,7.7,22.1,7.5,6.93,3.23,106.0,12.1,69.0
1,2,0=Blood Donor,32,m,38.5,70.3,18.0,24.7,3.9,11.17,4.80,74.0,15.6,76.5
2,3,0=Blood Donor,32,m,46.9,74.7,36.2,52.6,6.1,8.84,5.20,86.0,33.2,79.3
3,4,0=Blood Donor,32,m,43.2,52.0,30.6,22.6,18.9,7.33,4.74,80.0,33.8,75.7
4,5,0=Blood Donor,32,m,39.2,74.1,32.6,24.8,9.6,9.15,4.32,76.0,29.9,68.7


In [3]:
df.rename(columns={'Category': 'target'}, inplace=True)
df.loc[df["target"] == "0=Blood Donor", "target"] = "0"
df.loc[df["target"] == "0s=suspect Blood Donor", "target"] = "1"
df.loc[df["target"] == "1=Hepatitis", "target"] = "2"
df.loc[df["target"] == "2=Fibrosis", "target"] = "3"
df.loc[df["target"] == "3=Cirrhosis", "target"] = "4"

In [4]:
df = df.drop(df.columns[0], axis=1)
df.loc[df["Sex"] == "m", "Sex"] = "1"
df.loc[df["Sex"] == "f", "Sex"] = "0"
df = df.apply(pd.to_numeric, errors='coerce')
df.columns = df.columns.str.replace('[\[\]{}:",]', '', regex=True)
df.head()

,target,Age,Sex,ALB,ALP,ALT,AST,BIL,CHE,CHOL,CREA,GGT,PROT
0,0,32,1,38.5,52.5,7.7,22.1,7.5,6.93,3.23,106.0,12.1,69.0
1,0,32,1,38.5,70.3,18.0,24.7,3.9,11.17,4.80,74.0,15.6,76.5
2,0,32,1,46.9,74.7,36.2,52.6,6.1,8.84,5.20,86.0,33.2,79.3
3,0,32,1,43.2,52.0,30.6,22.6,18.9,7.33,4.74,80.0,33.8,75.7
4,0,32,1,39.2,74.1,32.6,24.8,9.6,9.15,4.32,76.0,29.9,68.7


In [11]:
from sklearn.model_selection import train_test_split

y = df['target']
x = df.drop(['target'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [14]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
X_train_t = scaler.transform(X_train)
X_test_t = scaler.transform(X_test)


In [27]:
columns = X_train.columns
X_train_t_df = pd.DataFrame(X_train_t, columns=columns)
X_test_t_df = pd.DataFrame(X_test_t, columns=columns)
X_train_t_df.head()

,Age,Sex,ALB,ALP,ALT,AST,BIL,CHE,CHOL,CREA,GGT,PROT
0,-0.835559,0.855800,-0.901069,0.160551,0.027678,-0.257913,-0.280688,-1.157568,-0.634309,-0.080611,-0.322796,-1.056130
1,-0.835559,0.855800,0.726659,-0.251822,-0.320453,-0.346239,1.817927,0.219224,-0.532370,-0.080611,-0.486144,0.667928
2,0.591500,0.855800,0.496654,-0.312741,-0.587920,-0.480152,-0.179486,-0.566162,-0.345483,-0.208413,-0.427632,0.012036
3,1.101164,-1.168497,-2.723417,-0.720429,0.477701,1.203739,-0.296667,-1.417786,-1.594232,-0.756136,-0.113126,-3.923313
4,-1.447156,-1.168497,0.125108,3.248666,-0.596411,-0.622613,-0.424502,0.039437,NaN,-0.610077,-0.247217,0.461790


In [37]:
nan_counts = X_train.isna().sum()
print("NaN counts by columns:")
print(nan_counts)

nan_counts = X_train_t_df.isna().sum()
print("NaN counts by columns:")
print(nan_counts)


NaN counts by columns:
Age      0
Sex      0
ALB      0
ALP     10
ALT      1
AST      0
BIL      0
CHE      0
CHOL     8
CREA     0
GGT      0
PROT     0
dtype: int64
NaN counts by columns:
Age     0
Sex     0
ALB     0
ALP     0
ALT     0
AST     0
BIL     0
CHE     0
CHOL    0
CREA    0
GGT     0
PROT    0
dtype: int64


In [42]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean')
X_train_imputed = pd.DataFrame(imputer.fit_transform(X_train_t_df), columns=X_train_t_df.columns)
X_test_imputed = pd.DataFrame(imputer.transform(X_test_t_df), columns=X_test_t_df.columns)


In [43]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


clf = RandomForestClassifier(random_state=42)


clf.fit(X_train_imputed, y_train)


y_pred = clf.predict(X_test_imputed)


accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)


print(f"Accuracy: {accuracy}")
print("\nConfusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(classification_rep)

Accuracy: 0.8617886178861789

Confusion Matrix:
[[96  0  0  0  0]
 [ 2  0  0  1  0]
 [ 5  0  2  1  1]
 [ 4  0  0  2  0]
 [ 2  0  0  1  6]]

Classification Report:
              precision    recall  f1-score   support

           0       0.88      1.00      0.94        96
           1       0.00      0.00      0.00         3
           2       1.00      0.22      0.36         9
           3       0.40      0.33      0.36         6
           4       0.86      0.67      0.75         9

    accuracy                           0.86       123
   macro avg       0.63      0.44      0.48       123
weighted avg       0.84      0.86      0.83       123



/home/urie/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/urie/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/urie/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [44]:
import pickle

with open('rfm.pkl', 'wb') as file:
    pickle.dump(clf, file)